# What Do You Listen to?

### Description

As someone who regularly attends concerts it is often that you are asked, What kind of music do you listen to? Typically I will respond with the name of the band that is playing and maybe one or two other similar artists if applicable. In the back of my head I know this is not the most accurate response but with today's microgenres and availability of music it can be hard to capture exactly what you listen to might mean.

I decided that as an exercise to familiarize myself a bit more with Tableau and data manipulation in Python, I would collect my entire history of saved Spotify music and try and answer a few questions.

What I found was not exactly what I expected.

### Contents

**Section 1:** Wrangling our data

**Section 2:** Analysis
       - What genres do you listen to?
       - How has your music taste changed over time?
       - Are these genres bogus hipster lingo or is there some actual classifcation behind them?
       - Based on your genre preferences, who will you like at this year's Coachella Music Festival?
    
**Section 3:** Wrap-up: Results, Limitations, and Improvements

## Section 1: Wrangling our Data

To get the data for this project we will look at all of my saved playlists. This is typically how I use Spotify; find tracks, albums, or even artist discographies I enjoy and drag them to a playlist. Unfortunately, Spotify does not make play count data easily available or else that would have been my preferred method of answering our questions, but the methodology I chose accurately represents how I use Spotify so it will have to do.

The code below is the high level process for how I wrangled the data. The Song and Playlist classes are available in the util directory of this repository if you want to see how it all works.

In [79]:
from util.playlist import Playlist
from util.util import create_auth_session

# authenticate w spotify
# NOTE: you need to add your own client id, secret, and redirect to the env
sp = create_auth_session()

# get list of playlists
user = "ahontz"
playlists = [Playlist(playlist['name'], playlist['id'], playlist['owner']['id']) for playlist in sp.user_playlists(limit=50, user="ahontz")["items"]]

# manually adding playlists not returned by query above
CoachElla = Playlist("coach ella", "spotify:user:ahontz:playlist:3RWmhIFFvlYj2FMiPdzZdM", "ahontz")
discover_yearly = Playlist("DiscoverYearly", "spotify:user:ahontz:playlist:7kRosMbZhZfucMBXqStcsG", "ahontz")
playlists.append(CoachElla)
playlists.append(discover_yearly)

# get tracklist and write to csv, this could take awhile
for playlist in playlists:
    # refresh our token in case this takes over an hour
    try:
        playlist.request_songs(sp)
    except:
        sp = create_auth_session()
        
    playlist.to_csv(playlist.name)


            You need to set your Spotify API credentials. You can do this by
            setting environment variables like so:

            export SPOTIPY_CLIENT_ID='your-spotify-client-id'
            export SPOTIPY_CLIENT_SECRET='your-spotify-client-secret'
            export SPOTIPY_REDIRECT_URI='your-app-redirect-url'

            Get your credentials at     
                https://developer.spotify.com/my-applications
        


SpotifyException: http status: 550, code:-1 - no credentials set

The rest of our data manipulation will be done in pandas (joining our playlist files and writing out to csv, converting fields to dates, deconstructing genres lists to get individual genre counts, conduction our ANOVA test, etc)

## Section 2: Analysis

### Question 1: What genres do you listen to?

**Measure:** Top 50% most frequently occuring genres in frequency tables for All Saved Music, All Saved Music from Jan 1 2016 onward, and my Top 100 most played songs of 2016.

**Comments:** This measure has little regard for play count as Spotify does not expose this with their API. I theoretically could have saved a ton of "indietronica" music, maybe listened to one or two tracks, and then gone on and listened to the same country song for the remainder of the year and our results would still be the same.

As mentioned before, this is not how I use Spotify so it shouldn't be of concern but just as a back up I do have the one insight into play count, my Top 100 most played songs of 2016, to give us some indication of what was actually used within my saved music.

**Result: ** As you can see in the table below, my tastes are pretty consistent across these different playlists. I would give the most weight to Top 100 when answering the question "What do you listen to" as it takes play count into effect.

**Possible Improvement:** To get these genre counts, I had to deconstruct the 'genres' field in our data. The genres field was most often a list of genres that when combined, would describe the artist's discography. For artists who have changed genres over time, for example someone like Taylor Swift who released a country album to gain popularity but quickly transitioned into being known for pop music, she would be equally represented as both country and pop although the majority of her music would fall under the pop category. Finding a way to measure genre at the album or track level could improve the insights we get from this result.

Weighting our micro genres would also more accurately describe our genre count. If an artist is described as [indietronica, chillwave, new rave] instead of adding one to the count of each genre, I could try and add 1/N, where N is the number of genre's describing the artist.

| Rank | All Music | Recent Music | Top 100 Last Year | 
|---|---|---|
| 1 |  indietronica | indietronica |  indietronica |
| 2 | indie rock | chillwave | indie pop |
| 3 | indie pop | wonky | indie rock |
| 4 | chillwave | indie pop | chillwave |
| 5 | neo-psychedelic | indie r&b | alternative dance |
| 6 | alternative dance | alternative dance |  indie r&b |
| 7 | shimmer pop | electronic | new rave |
| 8 | new rave | new rave  | alternative rock |
| 9 | indie r&b | indie rock | electronic |
| 10 | indie folk | neo-psychedelic | wonky |

In [11]:
%%HTML
<div class='tableauPlaceholder' id='viz1487101314035' style='position: relative'><noscript><a href='#'><img alt='What genres do you listen to? ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;3F&#47;3F8C2GG7M&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='path' value='shared&#47;3F8C2GG7M' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;3F&#47;3F8C2GG7M&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1487101314035');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='1020px';vizElement.style.height='1033px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

### Question 2: How have your tastes changed over time?

**Measure:** Select all genres that have 50 or more occurances in all of my saved music. Visually observe the rankings change between time groupings.

**Comments:** I created custom groupings as there was little saved music between 2013-2015 (guess I was too busy working) so we have defined our periods as 2011, 2012, 2013-2015, and 2016 onwards.

**Result:** Although we are relying on visual observation here, you can clearly see the difference in genre names in the 2011 grouping and the most recent. 2011 has a lot more indie and folk influence as those genres defined alternative music throughout the late 2000s and early 2010s. In our latest grouping indie folk does not even make the top 20; instead it has been overtaken by obscure electronic micro genres like escape room, vapor twitch, and ninja.

**Improvement:** If the data were available it would be nice to view the popularity of these genres (or the bands that define them) over this period. I would like to be able to learn if the popularization of music streaming services, or the general increase in music access, has shortened the popularity phase of music genres. I would also like to know who is naming these genres...

In [14]:
%%HTML
<div class='tableauPlaceholder' id='viz1487101473694' style='position: relative'><noscript><a href='#'><img alt='Genre Breakdown By Frequency - 2011 ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;ge&#47;genres_over_time&#47;GenreBreakdownoverTime&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='site_root' value='' /><param name='name' value='genres_over_time&#47;GenreBreakdownoverTime' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;ge&#47;genres_over_time&#47;GenreBreakdownoverTime&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1487101473694');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

### Question 3: Are these genres bogus or are there distinctions in their audio features?

**Measure:** Select 4 of my top ten genres that my intuition would tell me have distinct features, plot them side by side with their average audio features and observe the differences.

**Result:** In an ANOVA test comparing each genre's audio features, we find that acousticness, instrumentalness, danceability, and speechiness are all significant at the p=0.005 (0.5%) level, indicating that there exists a significant difference in the means for these features in this group. That does however suggest that valence and energy do not vary much between this particular group of genres.

Additionally, I was confused by the fact that alternative rock, was rated at the lowest average instrumentalness.

**Improvement:** Pick either a genre I do not listen to at all or one of my least listened genres and compare; hopefully there would be more significant **visual** changes there.

In [13]:
%%HTML
<div class='tableauPlaceholder' id='viz1487101402728' style='position: relative'><noscript><a href='#'><img alt='visualizing genres ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;ge&#47;genres_0&#47;visualizinggenres&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='site_root' value='' /><param name='name' value='genres_0&#47;visualizinggenres' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;ge&#47;genres_0&#47;visualizinggenres&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1487101402728');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

In [78]:
import pandas as pd
from scipy.stats import f_oneway

df = pd.read_csv("Genre_Blast.csv")
feature_list = [u'energy', u'liveness', u'acousticness', u'valence', u'instrumentalness', u'danceability', u'speechiness']
for feature in feature_list:

    altrock = df.loc[df["genre"] == "alternative rock", feature].fillna(df.loc[df["genre"] == "alternative rock", feature].mean())
    chillwave = df.loc[df["genre"] == "chillwave", feature].fillna(df.loc[df["genre"] == "chillwave", feature].mean())
    indietron = df.loc[df["genre"] == "indietronica", feature].fillna(df.loc[df["genre"] == "indietronica", feature].mean())
    wonky = df.loc[df["genre"] == "wonky", feature].fillna(df.loc[df["genre"] == "wonky", feature].mean())
    
    print "{}: {}".format(feature ,f_oneway(altrock, chillwave, indietron, wonky))
    print ""

energy: F_onewayResult(statistic=1.5286537772728881, pvalue=0.20499107395724531)

liveness: F_onewayResult(statistic=2.3324062028645458, pvalue=0.072214437483745619)

acousticness: F_onewayResult(statistic=4.9223881804651457, pvalue=0.0020599555348630094)

valence: F_onewayResult(statistic=0.41350714326715626, pvalue=0.74331503534095789)

instrumentalness: F_onewayResult(statistic=18.010258487073674, pvalue=1.4002789869114019e-11)

danceability: F_onewayResult(statistic=38.563380881301278, pvalue=1.9470562708390518e-24)

speechiness: F_onewayResult(statistic=96.325688512779223, pvalue=1.8258304655227106e-59)



### Question 4: Based on your genre preferences, who should you see at Coachella this year?

**Measure:** Return a list of artists who fall in one or more of my top ten genres. Deconstruct the track's genre list and count how many of the genres in the that list are in my top ten. Repeat this for each artist's top 10 songs adding 1 for each match. Sort the list descending.

**Comments:** This measure favors those who cannot be classified with just one genre. All artists are equally likely to appear (since the playlist is composed of the top 10 songs from each artist) but those with more genres are more likely to have a higher score.

Example: Lets say Radiohead has a genre list of [indietronica, indie rock]. This means at most, if all of their songs fit only these two genres, they can score a 20 (1 point for a song in my top ten genre, they have 10 songs in 2 genres). Now let's look at Crystal Castles, for the same 10 songs, they might have a genre list [new rave, wonky, electronic, alternative dance] which would score them 40 points. 

A score of 40 vs 20 doesn't mean that Crystal Castles is a better match, as we showed, they could just have higher potential to score as in this case.

**Result:** Whether or not the ordering or sorting is appropriate, the resulting list serves it's intended purpose. It has identified bands I already listen to that I am most excited to see (Real Estate, Future Islands, Empire of the Sun) and placed them toward the top of the list, while keeping those I am not really interested in at the bottom (Snakehips, Bicep, Jamie xx).

**Possible Improvement:** The best possible way I could improve recommendations for who I shoud see (i.e. who matches my tastes) would be to use my skills with machine learning to write an item by item classifier. During this project I have likely accumulated enough data to start training a classifier that could label each song in this Coachella set as a reccomended song or not. Since there are different genres that we can assume are generated by differences in the audio features Spotify computes during their audio signal processing, it might be best to build separate classifier for each genre (or form our own 'parent' genres). Let's look at the next question to see how different these genre's really are.

In [12]:
%%HTML
<div class='tableauPlaceholder' id='viz1487101360419' style='position: relative'><noscript><a href='#'><img alt='Coachella Artists Matching My Taste In Genre ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;co&#47;coachellaartistsinmygenres&#47;CoachellaArtistList&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='site_root' value='' /><param name='name' value='coachellaartistsinmygenres&#47;CoachellaArtistList' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;co&#47;coachellaartistsinmygenres&#47;CoachellaArtistList&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1487101360419');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

## Section 3: Wrap-up: Findings & Limitations

Through the course of this project I have examined which genres I listen (and have listened) to, looked at my change in taste over time, examined differences in some of my top genres, and even created a basic method for making artist recommendations.

However, as I have stated before, our findings are all based on my saved Spotify music.

This does not include:
- what I actually listened to (I may save every Radiohead album but I have listened to the SALES EP from SALES for hours on end)
- play count with friend's or public playlists
- play count on radio or daily mixes
- plays from general browsing or recommended artist hopping

To really answer some of these questions the best way possible, I would have need access to play history, which is accesible (unfortunately not retroactively) day forward with scrobbling to Last.Fm